In [1]:
import pandas as pd
import numpy as np
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import iplot, init_notebook_mode
import matplotlib.pyplot as plt
import seaborn as sns
import cufflinks
cufflinks.go_offline(connected=True)
init_notebook_mode(connected=True)
from sklearn.model_selection import cross_val_score
pd.set_option('display.max_row', 100)
pd.set_option('display.max_column', 150)

import warnings
warnings.filterwarnings('ignore')

# Import processing dataset

In [2]:
train_x = pd.read_csv("train_x.csv")
test_x = pd.read_csv("test_x.csv")
train_x_log = pd.read_csv("train_x_log.csv")
test_x_log = pd.read_csv("test_x_log.csv")
train_x_eng_with_KNN_TSNE_KMean = pd.read_csv("train_x_eng_with_KNN_TSNE_KMean.csv")
test_x_eng_with_KNN_TSNE_Kmean = pd.read_csv("test_x_eng_with_KNN_TSNE_Kmean.csv")
train_x_log_eng_with_KNN_TSNE_KMean = pd.read_csv("train_x_log_eng_with_KNN_TSNE_KMean.csv")
test_x_log_eng_with_KNN_TSNE_Kmean = pd.read_csv("test_x_log_eng_with_KNN_TSNE_Kmean.csv")
train_y_class = pd.read_csv("train_y_class.csv")
train_y_label = pd.read_csv("train_y_label.csv")

Submission = pd.read_csv("sampleSubmission.csv")

In [3]:
# Total dataset features to consider
print(train_x.shape)
print(test_x.shape)
print(train_x_log.shape)
print(test_x_log.shape)
print(train_x_eng_with_KNN_TSNE_KMean.shape)
print(test_x_eng_with_KNN_TSNE_Kmean.shape)
print(train_x_log_eng_with_KNN_TSNE_KMean.shape)
print(test_x_log_eng_with_KNN_TSNE_Kmean.shape)

# Label dataset
print(train_y_class.shape)
print(train_y_label.shape)

(61878, 93)
(144368, 93)
(61878, 93)
(144368, 93)
(61878, 227)
(144368, 227)
(61878, 227)
(144368, 227)
(61878, 9)
(61878, 1)


# 1. Random Forest Classifier, Grid Search with train_x, train_y_class

In [4]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

In [5]:
hyperparameters = {"criterion": ["gini"],
                   "max_depth": [5],   
                   "max_features": [0.5], 
                   "min_samples_split": [2],
                   "n_estimators": [100],
                   "random_state": [0], # Whether to use out-of-bag samples to estimate the generalization accuracy.
                  }

In [6]:
cls = RandomForestClassifier()
grid = GridSearchCV(cls,param_grid=hyperparameters,cv=4)
grid.fit(train_x, train_y_class)

GridSearchCV(cv=4, error_score='raise-deprecating',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'criterion': ['gini'], 'max_depth': [5], 'max_features': [0.5], 'min_samples_split': [2], 'n_estimators': [100], 'random_state': [0]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [7]:
best_params = grid.best_params_
best_score = grid.best_score_
best_rf = grid.best_estimator_
print(best_score)
print(best_rf)

0.039157697404570285
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=5, max_features=0.5, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
            oob_score=False, random_state=0, verbose=0, warm_start=False)


In [8]:
best_rf.score(train_x,train_y_class)

0.3705517308251721

# test_x Prediction for Ensemble Model

In [9]:
RF_test_x_pred = best_rf.predict_proba(test_x)

In [10]:
RF_test_x_pred = np.array(RF_test_x_pred)
RF_test_x_pred = RF_test_x_pred.reshape((144368,9,2))
RF_test_x_pred = RF_test_x_pred[:,:,0]
RF_test_x_pred = pd.DataFrame(RF_test_x_pred, columns = ["RF_test_x_pred_" + str(i) for i in range(1,10,1)])
print(RF_test_x_pred.shape)

(144368, 9)


In [11]:
Submission = pd.read_csv("sampleSubmission.csv")
Submission[['Class_1', 'Class_2', 'Class_3', 'Class_4', 'Class_5', 'Class_6',
       'Class_7', 'Class_8', 'Class_9']] = RF_test_x_pred

In [12]:
## Creating a Submission File to submit to Kaggle competition ##
Submission.to_csv("RF_test_x_pred.csv",index=False)

# train_x Prediction for Ensemble Model

In [13]:
RF_train_x_pred = best_rf.predict_proba(train_x)

In [14]:
RF_train_x_pred = np.array(RF_train_x_pred)
RF_train_x_pred = RF_train_x_pred.reshape((61878,9,2))
RF_train_x_pred = RF_train_x_pred[:,:,0]
RF_train_x_pred = pd.DataFrame(RF_train_x_pred, columns = ["RF_train_x_pred_" + str(i) for i in range(1,10,1)])
print(RF_train_x_pred.shape)

(61878, 9)


In [15]:
Submission = pd.read_csv("sampleSubmission.csv")
Submission[['Class_1', 'Class_2', 'Class_3', 'Class_4', 'Class_5', 'Class_6',
       'Class_7', 'Class_8', 'Class_9']] = RF_train_x_pred

In [16]:
## Creating a Submission File to submit to Kaggle competition ##
Submission.to_csv("RF_train_x_pred.csv",index=False)

# 2. Random Forest Classifier, Grid Search with train_x_log, train_y_class

In [17]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

In [18]:
hyperparameters = {"criterion": ["gini"],
                   "max_depth": [5],   
                   "max_features": [0.5], 
                   "min_samples_split": [2],
                   "n_estimators": [100],
                   "random_state": [0], # Whether to use out-of-bag samples to estimate the generalization accuracy.
                  }

In [19]:
cls = RandomForestClassifier()
grid = GridSearchCV(cls,param_grid=hyperparameters,cv=4)
grid.fit(train_x_log, train_y_class)

GridSearchCV(cv=4, error_score='raise-deprecating',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'criterion': ['gini'], 'max_depth': [5], 'max_features': [0.5], 'min_samples_split': [2], 'n_estimators': [100], 'random_state': [0]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [20]:
best_params = grid.best_params_
best_score = grid.best_score_
best_rf = grid.best_estimator_
print(best_score)
print(best_rf)

0.039157697404570285
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=5, max_features=0.5, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
            oob_score=False, random_state=0, verbose=0, warm_start=False)


In [21]:
best_rf.score(train_x_log,train_y_class)

0.3705517308251721

# test_x Prediction for Ensemble Model

In [22]:
RF_test_x_log_pred = best_rf.predict_proba(test_x_log)

In [23]:
RF_test_x_log_pred = np.array(RF_test_x_log_pred)
RF_test_x_log_pred = RF_test_x_log_pred.reshape((144368,9,2))
RF_test_x_log_pred = RF_test_x_log_pred[:,:,0]
RF_test_x_log_pred = pd.DataFrame(RF_test_x_log_pred, columns = ["RF_test_x_log_pred_" + str(i) for i in range(1,10,1)])
print(RF_test_x_log_pred.shape)

(144368, 9)


In [24]:
Submission = pd.read_csv("sampleSubmission.csv")
Submission[['Class_1', 'Class_2', 'Class_3', 'Class_4', 'Class_5', 'Class_6',
       'Class_7', 'Class_8', 'Class_9']] = RF_test_x_log_pred

In [25]:
## Creating a Submission File to submit to Kaggle competition ##
Submission.to_csv("RF_test_x_log_pred.csv",index=False)

# train_x Prediction for Ensemble Model

In [26]:
RF_train_x_log_pred = best_rf.predict_proba(train_x_log)

In [27]:
RF_train_x_log_pred = np.array(RF_train_x_log_pred)
RF_train_x_log_pred = RF_train_x_log_pred.reshape((61878,9,2))
RF_train_x_log_pred = RF_train_x_log_pred[:,:,0]
RF_train_x_log_pred = pd.DataFrame(RF_train_x_log_pred, columns = ["RF_train_x_log_pred_" + str(i) for i in range(1,10,1)])
print(RF_train_x_log_pred.shape)

(61878, 9)


In [28]:
Submission = pd.read_csv("sampleSubmission.csv")
Submission[['Class_1', 'Class_2', 'Class_3', 'Class_4', 'Class_5', 'Class_6',
       'Class_7', 'Class_8', 'Class_9']] = RF_train_x_log_pred

In [29]:
## Creating a Submission File to submit to Kaggle competition ##
Submission.to_csv("RF_train_x_log_pred.csv",index=False)

# 3. Random Forest Classifier, Grid Search with train_x_eng_with_KNN_TSNE_KMean, train_y_class

In [30]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

In [31]:
hyperparameters = {"criterion": ["gini"],
                   "max_depth": [6],   
                   "max_features": [0.9], 
                   "min_samples_split": [2],
                   "n_estimators": [100],
                   "random_state": [0], # Whether to use out-of-bag samples to estimate the generalization accuracy.
                  }

In [32]:
cls = RandomForestClassifier()
grid = GridSearchCV(cls,param_grid=hyperparameters,cv=4)
grid.fit(train_x_eng_with_KNN_TSNE_KMean, train_y_class)

GridSearchCV(cv=4, error_score='raise-deprecating',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'criterion': ['gini'], 'max_depth': [6], 'max_features': [0.9], 'min_samples_split': [2], 'n_estimators': [100], 'random_state': [0]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [33]:
best_params = grid.best_params_
best_score = grid.best_score_
best_rf = grid.best_estimator_
print(best_score)
print(best_rf)

0.29259187433336564
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=6, max_features=0.9, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
            oob_score=False, random_state=0, verbose=0, warm_start=False)


In [34]:
best_rf.score(train_x_eng_with_KNN_TSNE_KMean,train_y_class)

0.8051973237661204

# test_x Prediction for Ensemble Model

In [35]:
RF_test_x_eng_pred = best_rf.predict_proba(test_x_eng_with_KNN_TSNE_Kmean)

In [36]:
RF_test_x_eng_pred = np.array(RF_test_x_eng_pred)
RF_test_x_eng_pred = RF_test_x_eng_pred.reshape((144368,9,2))
RF_test_x_eng_pred = RF_test_x_eng_pred[:,:,0]
RF_test_x_eng_pred = pd.DataFrame(RF_test_x_eng_pred, columns = ["RF_test_x_eng_pred_" + str(i) for i in range(1,10,1)])
print(RF_test_x_eng_pred.shape)

(144368, 9)


In [37]:
Submission = pd.read_csv("sampleSubmission.csv")
Submission[['Class_1', 'Class_2', 'Class_3', 'Class_4', 'Class_5', 'Class_6',
       'Class_7', 'Class_8', 'Class_9']] = RF_test_x_eng_pred

In [38]:
## Creating a Submission File to submit to Kaggle competition ##
Submission.to_csv("RF_test_x_eng_pred.csv",index=False)

# train_x Prediction for Ensemble Model

In [39]:
RF_train_x_eng_pred = best_rf.predict_proba(train_x_eng_with_KNN_TSNE_KMean)

In [40]:
RF_train_x_eng_pred = np.array(RF_train_x_eng_pred)
RF_train_x_eng_pred = RF_train_x_eng_pred.reshape((61878,9,2))
RF_train_x_eng_pred = RF_train_x_eng_pred[:,:,0]
RF_train_x_eng_pred = pd.DataFrame(RF_train_x_eng_pred, columns = ["RF_train_x_eng_pred_" + str(i) for i in range(1,10,1)])
print(RF_train_x_eng_pred.shape)

(61878, 9)


In [41]:
Submission = pd.read_csv("sampleSubmission.csv")
Submission[['Class_1', 'Class_2', 'Class_3', 'Class_4', 'Class_5', 'Class_6',
       'Class_7', 'Class_8', 'Class_9']] = RF_train_x_eng_pred

In [42]:
## Creating a Submission File to submit to Kaggle competition ##
Submission.to_csv("RF_train_x_eng_pred.csv",index=False)

# 4. Random Forest Classifier, Grid Search with train_x_log_eng_with_KNN_TSNE_KMean, train_y_class

In [43]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

In [44]:
hyperparameters = {"criterion": ["gini"],
                   "max_depth": [6],   
                   "max_features": [0.9], 
                   "min_samples_split": [2],
                   "n_estimators": [100],
                   "random_state": [0], # Whether to use out-of-bag samples to estimate the generalization accuracy.
                  }

In [45]:
cls = RandomForestClassifier()
grid = GridSearchCV(cls,param_grid=hyperparameters,cv=4)
grid.fit(train_x_log_eng_with_KNN_TSNE_KMean, train_y_class)

GridSearchCV(cv=4, error_score='raise-deprecating',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'criterion': ['gini'], 'max_depth': [6], 'max_features': [0.9], 'min_samples_split': [2], 'n_estimators': [100], 'random_state': [0]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [46]:
best_params = grid.best_params_
best_score = grid.best_score_
best_rf = grid.best_estimator_
print(best_score)
print(best_rf)

0.29259187433336564
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=6, max_features=0.9, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
            oob_score=False, random_state=0, verbose=0, warm_start=False)


In [47]:
best_rf.score(train_x_log_eng_with_KNN_TSNE_KMean,train_y_class)

0.8051973237661204

# test_x Prediction for Ensemble Model

In [48]:
RF_test_x_log_eng_pred = best_rf.predict_proba(test_x_log_eng_with_KNN_TSNE_Kmean)

In [49]:
RF_test_x_log_eng_pred = np.array(RF_test_x_log_eng_pred)
RF_test_x_log_eng_pred = RF_test_x_log_eng_pred.reshape((144368,9,2))
RF_test_x_log_eng_pred = RF_test_x_log_eng_pred[:,:,0]
RF_test_x_log_eng_pred = pd.DataFrame(RF_test_x_log_eng_pred, columns = ["RF_test_x_log_eng_pred_" + str(i) for i in range(1,10,1)])
print(RF_test_x_log_eng_pred.shape)

(144368, 9)


In [50]:
Submission = pd.read_csv("sampleSubmission.csv")
Submission[['Class_1', 'Class_2', 'Class_3', 'Class_4', 'Class_5', 'Class_6',
       'Class_7', 'Class_8', 'Class_9']] = RF_test_x_log_eng_pred

In [51]:
## Creating a Submission File to submit to Kaggle competition ##
Submission.to_csv("RF_test_x_log_eng_pred.csv",index=False)

# train_x Prediction for Ensemble Model

In [52]:
RF_train_x_log_eng_pred = best_rf.predict_proba(train_x_log_eng_with_KNN_TSNE_KMean)

In [53]:
RF_train_x_log_eng_pred = np.array(RF_train_x_log_eng_pred)
RF_train_x_log_eng_pred = RF_train_x_log_eng_pred.reshape((61878,9,2))
RF_train_x_log_eng_pred = RF_train_x_log_eng_pred[:,:,0]
RF_train_x_log_eng_pred = pd.DataFrame(RF_train_x_log_eng_pred, columns = ["RF_train_x_log_eng_pred_" + str(i) for i in range(1,10,1)])
print(RF_train_x_log_eng_pred.shape)

(61878, 9)


In [54]:
Submission = pd.read_csv("sampleSubmission.csv")
Submission[['Class_1', 'Class_2', 'Class_3', 'Class_4', 'Class_5', 'Class_6',
       'Class_7', 'Class_8', 'Class_9']] = RF_train_x_log_eng_pred

In [55]:
## Creating a Submission File to submit to Kaggle competition ##
Submission.to_csv("RF_train_x_log_eng_pred.csv",index=False)